In [1]:
import calendar
import pandas as pd
import numpy as np
from datetime import date, timedelta
from sqlalchemy import create_engine
from math import ceil, floor, trunc

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
con_lite = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

pd.set_option('display.float_format','{:,.2f}'.format)
pd.set_option('max_rows',None)

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"

today = date.today()
today

datetime.date(2022, 2, 14)

### Today = Last Closed Business Day

In [2]:
#today = today - timedelta(days=3)
today

datetime.date(2022, 2, 14)

### Restart and run all cells

In [3]:
def number_of_spread(x):
    maxp = x[1]
    minp = x[5]
    div = 0.0
    gap = 0.0
    spd = 0
    spd1 = 0
    spd2 = 0
    if (maxp > minp): 
        mult = 1
    else:
    	tmpp = minp
    	minp = maxp
    	maxp = tmpp
    	mult = -1
        
    gap = round(maxp - minp,2)   
    
    if (maxp > 400.00):
        div = 2.0
        boundary = div * 200
        if ((minp < boundary)):
            spd1 = trunc((boundary - minp) / 1.00)
            spd2 = trunc(round(((maxp - boundary) / div),2))
            spd = spd1 + spd2
        else:
            spd = trunc(gap / div) 
    elif (maxp > 200.00):
        div = 1.0
        boundary = div * 200
        if (minp < boundary):
            spd1 = trunc((boundary - minp) / 0.50)
            spd2 = trunc(round(((maxp - boundary) / div),2))
            spd = spd1 + spd2
        else:
            spd = trunc(gap / div) 
    elif (maxp > 100.00):
        div = 0.50
        boundary = div * 200
        if (minp < boundary):
            spd1 = trunc((boundary - minp) / 0.25)
            spd2 = trunc(round(((maxp - boundary) / div),2))
            spd = spd1 + spd2
        else:
            spd = trunc(gap / div) 
    elif (maxp > 25.00):
        div = 0.25
        boundary = div * 100
        if (minp < boundary):
            spd1 = trunc((boundary - minp) / 0.10)
            spd2 = trunc(round(((maxp - boundary) / div),2))
            spd = spd1 + spd2
        else:
            spd = trunc(gap / div)
    elif (maxp > 10.00):
        div = 0.10
        boundary = div * 100
        if (minp < boundary):
            spd1 = ceil((boundary - minp) / 0.05)
            spd2 = trunc(round(((maxp - boundary) / div),2))
            spd = spd1 + spd2
        else:
            div = 0.10            
            spd = ceil(gap / div)
    elif (maxp > 5.00):
        div = 0.05
        boundary = div * 100
        if (minp < boundary):
            spd1 = trunc((boundary - minp) / 0.02)
            spd2 = trunc(round(((maxp - boundary) / div),2))
            spd = spd1 + spd2           
        else:
            div = 0.05            
            spd = ceil(gap / div)
    elif (maxp > 2.00):
        div = 0.02
        boundary = div * 100
        if (minp < boundary):
            spd1 = trunc((boundary - minp) / 0.01)
            spd2 = trunc((maxp - boundary) / div)
            spd = spd1 + spd2
        else:          
            spd = (gap / div)
    else:
        div = 0.01
        boundary = div * 100        
        spd = trunc(gap / div) 
        
    return spd * mult

In [4]:
cols = 'name spd target bl qty fm_price to_price ttl min min52 css_price status amount fm_date to_date days'.split()

In [5]:
format_dict = {
               'fm_price':'{:.2f}','to_price':'{:.2f}','price':'{:.2f}',    
               'max_price':'{:.2f}','min_price':'{:.2f}',
               'target':'{:.2f}','max':'{:.2f}','max52':'{:.2f}',
               'min':'{:.2f}','min52':'{:.2f}',
               'css_price':'{:.2f}','amount':'{:,.2f}',  
               'unit_cost':'{:.2f}','cost_amt':'{:,.2f}',
               'dividend':'{:.2f}','dividend_amount':'{:,.2f}',    
               'cost_amt':'{:,.2f}','div_amt':'{:,.2f}','cost_pct':'{:.2f}', 
               'buy_target':'{:.2f}','sell_target':'{:.2f}',
               'volume':'{:,.2f}','beta':'{:.2f}','cost':'{:,.2f}',
               'qty':'{:,}','available_qty':'{:,}','shares':'{:,}'}
pd.set_option("display.max_rows", None)

In [6]:
sql = '''
SELECT T.name,buy_target AS target, bl, T.qty,
fm_price,to_price,  ttl_spread AS ttl, A.min_price AS min, 
T.min_price AS min52, consensus_price AS css_price, buy_target * T.qty AS amount, T.status,
FM_date, to_date, days
FROM stocks T
JOIN sales A
ON T.name = A.name
LEFT JOIN consensus C
ON T.name = C.name
WHERE T.status IN ('B','O')
AND buy_target > 0
AND to_date = '%s'
'''
sql = sql % today
print(sql)

df_stock = pd.read_sql(sql, con_lite)
df_stock.sort_values(['name'],ascending=[True]).style.format(format_dict)


SELECT T.name,buy_target AS target, bl, T.qty,
fm_price,to_price,  ttl_spread AS ttl, A.min_price AS min, 
T.min_price AS min52, consensus_price AS css_price, buy_target * T.qty AS amount, T.status,
FM_date, to_date, days
FROM stocks T
JOIN sales A
ON T.name = A.name
LEFT JOIN consensus C
ON T.name = C.name
WHERE T.status IN ('B','O')
AND buy_target > 0
AND to_date = '2022-02-14'



,name,target,bl,qty,fm_price,to_price,ttl,min,min52,css_price,amount,status,fm_date,to_date,days
0,ASP,3.72,0,"30,000",3.84,3.78,-3,-0.52,2.12,4.40,"111,600.00",O,2022-02-09,2022-02-14,3
1,BGC,10.00,0,"10,000",10.60,10.40,-2,-0.94,9.20,13.30,"100,000.00",O,2022-02-08,2022-02-14,4
12,CPNCG,12.40,0,"10,000",12.70,12.40,-3,12.40,10.40,nan,"124,000.00",B,2022-02-11,2022-02-14,1
2,KKP,68.00,0,"3,000",72.00,70.25,-7,-1.74,49.75,78.42,"204,000.00",O,2022-02-09,2022-02-14,3
3,MEGA,43.00,1,"4,500",45.75,43.25,-10,-3.28,32.25,55.75,"193,500.00",O,2022-02-08,2022-02-14,4
11,NOBLE,5.85,0,"21,000",6.30,6.20,-2,6.20,5.90,7.30,"122,850.00",B,2022-02-11,2022-02-14,1
13,RATCH,42.75,0,"3,000",46.00,45.50,-2,45.25,42.00,0.00,"128,250.00",B,2022-02-11,2022-02-14,1
4,ROJNA,6.30,0,"18,000",6.60,6.50,-2,-1.52,4.56,8.05,"113,400.00",O,2022-02-10,2022-02-14,2
5,SCC,374.00,0,300,391.00,383.00,-8,-1.02,364.00,472.00,"112,200.00",B,2022-02-09,2022-02-14,3
6,SINGER,39.25,4,"1,500",48.00,42.00,-24,-9.90,28.25,56.56,"58,875.00",B,2022-02-01,2022-02-14,9


In [7]:
my_array = df_stock.to_numpy()
my_array

array([['ASP', 3.72, 0, 30000, 3.84, 3.78, -3, -0.52, 2.12, 4.4,
        111600.0, 'O', '2022-02-09', '2022-02-14', 3],
       ['BGC', 10.0, 0, 10000, 10.6, 10.4, -2, -0.94, 9.2, 13.3,
        100000.0, 'O', '2022-02-08', '2022-02-14', 4],
       ['KKP', 68.0, 0, 3000, 72.0, 70.25, -7, -1.74, 49.75, 78.42,
        204000.0, 'O', '2022-02-09', '2022-02-14', 3],
       ['MEGA', 43.0, 1, 4500, 45.75, 43.25, -10, -3.28, 32.25, 55.75,
        193500.0, 'O', '2022-02-08', '2022-02-14', 4],
       ['ROJNA', 6.3, 0, 18000, 6.6, 6.5, -2, -1.52, 4.56, 8.05,
        113400.0, 'O', '2022-02-10', '2022-02-14', 2],
       ['SCC', 374.0, 0, 300, 391.0, 383.0, -8, -1.02, 364.0, 472.0,
        112200.0, 'B', '2022-02-09', '2022-02-14', 3],
       ['SINGER', 39.25, 4, 1500, 48.0, 42.0, -24, -9.9, 28.25, 56.56,
        58875.0, 'B', '2022-02-01', '2022-02-14', 9],
       ['STA', 30.25, 3, 2500, 31.75, 32.0, 1, 0.79, 28.0, 37.3, 75625.0,
        'B', '2022-02-08', '2022-02-14', 4],
       ['SYNEX', 24.5, 

In [8]:
tmp = (np.apply_along_axis(number_of_spread, axis=1, arr=my_array))
tmp

array([ -3.,  -4.,  -9.,  -1.,  -4.,  -9., -11.,  -7.,  -3.,  -4.,  -3.,
        -7.,   0., -11., -16.,  -7.])

In [9]:
df_stock['spd'] = tmp.tolist()
df_stock['spd'] = df_stock['spd'].astype(int)
df_stock.style.format(format_dict)

,name,target,bl,qty,fm_price,to_price,ttl,min,min52,css_price,amount,status,fm_date,to_date,days,spd
0,ASP,3.72,0,"30,000",3.84,3.78,-3,-0.52,2.12,4.40,"111,600.00",O,2022-02-09,2022-02-14,3,-3
1,BGC,10.00,0,"10,000",10.60,10.40,-2,-0.94,9.20,13.30,"100,000.00",O,2022-02-08,2022-02-14,4,-4
2,KKP,68.00,0,"3,000",72.00,70.25,-7,-1.74,49.75,78.42,"204,000.00",O,2022-02-09,2022-02-14,3,-9
3,MEGA,43.00,1,"4,500",45.75,43.25,-10,-3.28,32.25,55.75,"193,500.00",O,2022-02-08,2022-02-14,4,-1
4,ROJNA,6.30,0,"18,000",6.60,6.50,-2,-1.52,4.56,8.05,"113,400.00",O,2022-02-10,2022-02-14,2,-4
5,SCC,374.00,0,300,391.00,383.00,-8,-1.02,364.00,472.00,"112,200.00",B,2022-02-09,2022-02-14,3,-9
6,SINGER,39.25,4,"1,500",48.00,42.00,-24,-9.90,28.25,56.56,"58,875.00",B,2022-02-01,2022-02-14,9,-11
7,STA,30.25,3,"2,500",31.75,32.00,1,0.79,28.00,37.30,"75,625.00",B,2022-02-08,2022-02-14,4,-7
8,SYNEX,24.50,1,"3,000",28.75,24.80,-17,-8.70,16.50,32.19,"73,500.00",B,2022-02-07,2022-02-14,5,-3
9,TQM,45.00,6,"4,500",47.00,46.00,-4,-1.60,44.50,96.25,"202,500.00",O,2022-02-09,2022-02-14,3,-4


In [10]:
df_stock[cols].style.format(format_dict)

,name,spd,target,bl,qty,fm_price,to_price,ttl,min,min52,css_price,status,amount,fm_date,to_date,days
0,ASP,-3,3.72,0,"30,000",3.84,3.78,-3,-0.52,2.12,4.40,O,"111,600.00",2022-02-09,2022-02-14,3
1,BGC,-4,10.00,0,"10,000",10.60,10.40,-2,-0.94,9.20,13.30,O,"100,000.00",2022-02-08,2022-02-14,4
2,KKP,-9,68.00,0,"3,000",72.00,70.25,-7,-1.74,49.75,78.42,O,"204,000.00",2022-02-09,2022-02-14,3
3,MEGA,-1,43.00,1,"4,500",45.75,43.25,-10,-3.28,32.25,55.75,O,"193,500.00",2022-02-08,2022-02-14,4
4,ROJNA,-4,6.30,0,"18,000",6.60,6.50,-2,-1.52,4.56,8.05,O,"113,400.00",2022-02-10,2022-02-14,2
5,SCC,-9,374.00,0,300,391.00,383.00,-8,-1.02,364.00,472.00,B,"112,200.00",2022-02-09,2022-02-14,3
6,SINGER,-11,39.25,4,"1,500",48.00,42.00,-24,-9.90,28.25,56.56,B,"58,875.00",2022-02-01,2022-02-14,9
7,STA,-7,30.25,3,"2,500",31.75,32.00,1,0.79,28.00,37.30,B,"75,625.00",2022-02-08,2022-02-14,4
8,SYNEX,-3,24.50,1,"3,000",28.75,24.80,-17,-8.70,16.50,32.19,B,"73,500.00",2022-02-07,2022-02-14,5
9,TQM,-4,45.00,6,"4,500",47.00,46.00,-4,-1.60,44.50,96.25,O,"202,500.00",2022-02-09,2022-02-14,3


In [11]:
buy_alert = df_stock[cols]
buy_alert.set_index(['name'],inplace=True)
buy_alert.sort_values(['spd'],ascending=[False]).style.format(format_dict)

,spd,target,bl,qty,fm_price,to_price,ttl,min,min52,css_price,status,amount,fm_date,to_date,days
name,,,,,,,,,,,,,,,
CPNCG,0,12.40,0,"10,000",12.70,12.40,-3,12.40,10.40,nan,B,"124,000.00",2022-02-11,2022-02-14,1
MEGA,-1,43.00,1,"4,500",45.75,43.25,-10,-3.28,32.25,55.75,O,"193,500.00",2022-02-08,2022-02-14,4
ASP,-3,3.72,0,"30,000",3.84,3.78,-3,-0.52,2.12,4.40,O,"111,600.00",2022-02-09,2022-02-14,3
SYNEX,-3,24.50,1,"3,000",28.75,24.80,-17,-8.70,16.50,32.19,B,"73,500.00",2022-02-07,2022-02-14,5
TISCO,-3,98.00,1,"2,000",99.00,98.75,-1,98.25,85.00,110.80,B,"196,000.00",2022-02-11,2022-02-14,1
BGC,-4,10.00,0,"10,000",10.60,10.40,-2,-0.94,9.20,13.30,O,"100,000.00",2022-02-08,2022-02-14,4
ROJNA,-4,6.30,0,"18,000",6.60,6.50,-2,-1.52,4.56,8.05,O,"113,400.00",2022-02-10,2022-02-14,2
TQM,-4,45.00,6,"4,500",47.00,46.00,-4,-1.60,44.50,96.25,O,"202,500.00",2022-02-09,2022-02-14,3
STA,-7,30.25,3,"2,500",31.75,32.00,1,0.79,28.00,37.30,B,"75,625.00",2022-02-08,2022-02-14,4


In [12]:
file_name = 'buy-alert.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

buy_alert.sort_values(['spd','name'],ascending=[False,True]).to_csv(output_file)
buy_alert.sort_values(['spd','name'],ascending=[False,True]).to_csv(data_file)
buy_alert.sort_values(['spd','name'],ascending=[False,True]).to_csv(box_file)
buy_alert.sort_values(['spd','name'],ascending=[False,True]).to_csv(one_file)